# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [1]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
data.info()
data.head(60)
# print('Количество дубликатов до:', data.duplicated().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**Вывод**
В таблице с данными мы видим 12 стобцов. В столбцах **"day_employed"** и **"total_income"** есть пропущенные значения типа **"NaN"**. От них необходимо избавиться. Пропуски в этих двух столбцах соответствуют друг другу в строке, возможно люди работали неофициально и не могут подтвердить доход и стаж. В трудовом стаже даны отрицательные значения. В столбце **education** заметен разный регистр.

## Шаг 2. Предобработка данных

### Обработка пропусков

In [2]:
print(data.isna().sum())
print()

# посчитаем медиаы
total_income_median = data['total_income'].median()
days_employed_median = data['days_employed'].median()
print('Средний трудовой стаж:', days_employed_median)
print('медианная зарплата:', total_income_median)
print()

# здесь я сначала заменил все пропущенные значения общей медианой, но после захотел сделать по каждой категории отдельно
# data['total_income'] = data['total_income'].fillna(total_income_median)

# заменим пропущенные значения в трудовом стаже на медиану и применим метод abs к стобцу, 
# чтобы избавиться от отрицательных значений
data['days_employed'] = data['days_employed'].fillna(days_employed_median)
data['days_employed'] = data['days_employed'].abs()


children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Средний трудовой стаж: -1203.369528770489
медианная зарплата: 145017.93753253992



In [3]:
# узнаем уникальные типы занятости для подсчета среди них медины
print(data['income_type'].unique())
print()

# узнаю отдельно медиану по каждой группе и записываю их, для проверки и вдруг пригодятся
employee_median = data[data['income_type'] == 'сотрудник']['total_income'].median()
print('Медиана зарплаты среди сотрудников:', employee_median)
retiree_median = data[data['income_type'] == 'пенсионер']['total_income'].median()
print('Медиана зарплаты среди пенсионеров:', retiree_median)
companion_median = data[data['income_type'] == 'компаньон']['total_income'].median()
print('Медиана зарплаты среди компаньонов:', companion_median)
civil_servant_median = data[data['income_type'] == 'госслужащий']['total_income'].median()
print('Медиана зарплаты среди госслужащих:', civil_servant_median)

unemployed_median = data[data['income_type'] == 'безработный']['total_income'].median()
print('Медиана зарплаты среди безработных:', unemployed_median)
entrepreneur_median = data[data['income_type'] == 'предприниматель']['total_income'].median()
print('Медиана зарплаты среди предпринимателей:', entrepreneur_median)
student_median = data[data['income_type'] == 'студент']['total_income'].median()
print('Медиана зарплаты среди студентов:', student_median)
on_maternity_leave_median = data[data['income_type'] == 'в декрете']['total_income'].median()
print('Медиана зарплаты среди декретников:', on_maternity_leave_median)

# заполняем пропуски в столбце, группируя по второму столбцу и применяем функцию для подсчета медианы
data['total_income'] = data.groupby('income_type')['total_income'].apply(lambda x: x.fillna(x.median()))

# print(data['total_income'].head(50))
print()
print(data.isna().sum())
print()

['сотрудник' 'пенсионер' 'компаньон' 'госслужащий' 'безработный'
 'предприниматель' 'студент' 'в декрете']

Медиана зарплаты среди сотрудников: 142594.39684740017
Медиана зарплаты среди пенсионеров: 118514.48641164352
Медиана зарплаты среди компаньонов: 172357.95096577113
Медиана зарплаты среди госслужащих: 150447.9352830068
Медиана зарплаты среди безработных: 131339.7516762103
Медиана зарплаты среди предпринимателей: 499163.1449470857
Медиана зарплаты среди студентов: 98201.62531401133
Медиана зарплаты среди декретников: 53829.13072905995

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64



**Вывод** узнаем кол-во пропусков. Их порядка 10% для каждово столбца. Просто так удалять строки с пропусками нельзя, мы навярняка потеряем много информации по другим столбцам. Общий трудовой стаж и ежемесячный доход количественные переменные, заполним их характерными велечинами. Я выбрал медиану и для одного, и для другого столбца, т.к. в них есть сильно выдающиеся велечины, которые сильно искажают результаты анализа. Медиану зарплат расчитал отдельно для каждого типа занятости. Особенно странные данные про трудовой стаж по сотни тысяч дней.

Пропуски в этих двух столбцах соответствуют друг другу в строке, возможно люди работали неофициально и не могут подтвердить доход и стаж и пропускали их заполнение.

### Замена типа данных

In [4]:
# data.info()
# print(data.head(10))
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')

# используем подтипы для уменьшения объёма памяти
data['dob_years'] = data['dob_years'].astype('int8')
data['children'] = data['children'].astype('int8')
data['education_id'] = data['education_id'].astype('int8')
data['debt'] = data['debt'].astype('int8')
data['family_status_id'] = data['family_status_id'].astype('int8')
# print(data.head(10))
print(data.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int8
days_employed       21525 non-null int64
dob_years           21525 non-null int8
education           21525 non-null object
education_id        21525 non-null int8
family_status       21525 non-null object
family_status_id    21525 non-null int8
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int8
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(2), int8(5), object(5)
memory usage: 1.3+ MB
None


**Вывод**
Т.к. в таблице в столбах трудовой стаж и зарплата даны не целые числа, имеют вещественный тип(как это человек отработал одну десятитысячную дня и получил часть копейки?), сами данные заменены на целочисленные. Использовал при этом метод astype('int'), так как необходимо было указать конкретный тип значений. Так же для оптимизации заполнения памяти в некоторых столбцах использовал подтип данных.

### Обработка дубликатов

In [5]:
# data['education'].value_counts()
data['education'] = data['education'].str.lower()
data['education'].value_counts()

# data['family_status'].value_counts()
# data['income_type'].value_counts()

# print(data.sort_values(by='days_employed', ascending=False).head(50))
print('Количество дубликатов до:', data.duplicated().sum())

data = data.drop_duplicates().reset_index(drop=True)
print('Количество дубликатов после:', data.duplicated().sum())

Количество дубликатов до: 71
Количество дубликатов после: 0


**Вывод** Посмотрим уникальные значения в некоторых столбцак. В столбце **"education"** отмечено, что некоторые значения отличаются только регистром. Простое приведение к нижнему регистру позволило избавиться от лишних. Дальше поищем дубликаты. Их оказалось 71 на 21 с половиной тысяч строк. Даже если перемножить только доход и трудовой стаж в днях, мы получим огромное количество возможных комбинаций и появление одинаковых людей маловероятно. Удалим их.

Вероятная причина появления дубликатов: после заполнения пропусков дубликатов стало несколько больше. Около 50 дубликатов было изначально и скорее всего они так же связы с пропусками данных в двух столбцах, что позволило появиться одинаковым данным.

### Лемматизация

In [6]:
# print(data['purpose'].value_counts())

# открываем библиотеку pymystem3
from pymystem3 import Mystem
m = Mystem()

# создаём функцию, по которой выделим леммы
def lemmatization(row):
    lemmas = ' '.join(m.lemmatize(row))
    return lemmas

# создадим новый столбец с леммами
data['lemmas'] = data['purpose'].apply(lemmatization)

# выделим уникальные значения из этого столбцаю Метод 'unique' выдаёт ошибку, в функцию добавили метод 'join'
print(data['lemmas'].unique())


['покупка   жилье \n' 'приобретение   автомобиль \n'
 'дополнительный   образование \n' 'сыграть   свадьба \n'
 'операция   с   жилье \n' 'образование \n' 'на   проведение   свадьба \n'
 'покупка   жилье   для   семья \n' 'покупка   недвижимость \n'
 'покупка   коммерческий   недвижимость \n'
 'покупка   жилой   недвижимость \n'
 'строительство   собственный   недвижимость \n' 'недвижимость \n'
 'строительство   недвижимость \n'
 'на   покупка   подержать   автомобиль \n'
 'на   покупка   свой   автомобиль \n'
 'операция   с   коммерческий   недвижимость \n'
 'строительство   жилой   недвижимость \n' 'жилье \n'
 'операция   со   свой   недвижимость \n' 'автомобиль \n'
 'заниматься   образование \n' 'сделка   с   подержанный   автомобиль \n'
 'получение   образование \n' 'свадьба \n'
 'получение   дополнительный   образование \n' 'покупка   свой   жилье \n'
 'операция   с   недвижимость \n' 'получение   высокий   образование \n'
 'свой   автомобиль \n' 'сделка   с   автомобиль \n'
 'про

**Вывод**
после лемматизации и выделение уникальных значений можно выделить несколько групп: автомобиль, образование, свадьба, жилая недвижимость, коммерческая недвижимость и строительство

### Категоризация данных

In [7]:
# print(data.info())
print(data['children'].unique())
data['children'] = data['children'].abs()
print(data['children'].unique())
# разобьём семьи по количеству детей
def children_status(children):
    if children == 0:
            return 'Бездетные'
    if children == 1:
            return '1_ребенок'
    if children == 2:
            return '2_ребенка'
    if children == 3:
            return '3_ребенка'
    if children == 4:
            return '4_ребенка'
    if children == 5:
            return '5_детей'
    return 'Табор'

#Применим метод и проверим результат
data['children_status'] = data['children'].apply(children_status)
print(data['children_status'].head(15))
print()
# print(data['total_income'].unique())

[ 1  0  3  2 -1  4 20  5]
[ 1  0  3  2  4 20  5]
0     1_ребенок
1     1_ребенок
2     Бездетные
3     3_ребенка
4     Бездетные
5     Бездетные
6     Бездетные
7     Бездетные
8     2_ребенка
9     Бездетные
10    2_ребенка
11    Бездетные
12    Бездетные
13    Бездетные
14    Бездетные
Name: children_status, dtype: object



В столбце количество детей нашлись отрицательные значения. Возможно ошибка при заполнении. Так же, как и в столбце "трудовой стаж" применили метод **abs**, чтобы от них изменить и сохранить

In [8]:
# print(data.sort_values(by='total_income')['total_income'].head(50))
# разобьём доходы на четыре категории отталкиваясь от ранее полученой медианы
def level_income(level):
    if level < 145017 * 0.7:
            return 'Низкие'
    if  145017 * 0.7 <= level < 145017 * 1.1:
            return 'Средние'
    if 145017 * 1.1 <= level < 145017 * 1.6:
            return 'Выше среднего'
    return 'Высокие'

#Применим метод и проверим результат
data['level_income'] = data['total_income'].apply(level_income)
print(data.loc[:, ['total_income', 'level_income']].head(15))

    total_income   level_income
0         253875        Высокие
1         112080        Средние
2         145885        Средние
3         267628        Высокие
4         158616        Средние
5         255763        Высокие
6         240525        Высокие
7         135823        Средние
8          95856         Низкие
9         144425        Средние
10        113943        Средние
11         77069         Низкие
12        118514        Средние
13        130458        Средние
14        165127  Выше среднего


In [9]:
# создадим цикл, чтобы выделить цели на кредит
for row in data['lemmas']:
    if 'жилье' in row:
        data['lemmas'] = data['lemmas'].replace(row, 'ипотека') 

    if 'автомобиль' in row:
        data['lemmas'] = data['lemmas'].replace(row, 'автокредит') 

    if 'образование' in row:
        data['lemmas'] = data['lemmas'].replace(row, 'образование') 

    if 'свадьба' in row:
        data['lemmas'] = data['lemmas'].replace(row, 'свадьба') 

    if 'жилой' in row:
        data['lemmas'] = data['lemmas'].replace(row, 'ипотека') 

    if 'строительство' in row:
        data['lemmas'] = data['lemmas'].replace(row, 'строительство')    

    if 'коммерческий' in row:
        data['lemmas'] = data['lemmas'].replace(row, 'коммерция') 

    if 'недвижимость' in row:
        data['lemmas'] = data['lemmas'].replace(row, 'ипотека')
        
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemmas,children_status,level_income
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,ипотека,1_ребенок,Высокие
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автокредит,1_ребенок,Средние
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,ипотека,Бездетные,Средние
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,3_ребенка,Высокие
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,Бездетные,Средние
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,ипотека,Бездетные,Высокие
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,ипотека,Бездетные,Высокие
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,образование,Бездетные,Средние
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,свадьба,2_ребенка,Низкие
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,ипотека,Бездетные,Средние


**Вывод** Данные категоризированы по уровню дохода, ведь от разного достатка люди могут по разному выплачивать кредит, наличия и количества детей, содержание иждивенцов не простое занятие. Так же цели на кредит приведены к единому стандарту, где выделены основные категории, столбец решил не добавлять к каждой категории.

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [10]:
# создадим сводную таблицу для оценки задержек при наличии детей
data_pivot_children = data.pivot_table(index = ['children_status'], columns = 'debt', values = 'purpose', aggfunc='count')
print(data_pivot_children)
print()

# появились пропущенные значения после подсчёта, видимо задержек по выплатам с 5 детьми не было
# заменим это значение на ноль
data_pivot_children = data_pivot_children.fillna(0)

# заменим названия столбцов, а то питон на них ругается при подсчете
data_pivot_children.set_axis(['no','yes'],axis='columns',inplace=True)
data_pivot_children['percentage of delay'] = data_pivot_children['yes'] / data_pivot_children['no']

data_pivot_children.sort_values(by='percentage of delay')

debt                   0       1
children_status                 
1_ребенок         4410.0   445.0
2_ребенка         1858.0   194.0
3_ребенка          303.0    27.0
4_ребенка           37.0     4.0
5_детей              9.0     NaN
Бездетные        13028.0  1063.0
Табор               68.0     8.0



,no,yes,percentage of delay
children_status,,,
5_детей,9.0,0.0,0.000000
Бездетные,13028.0,1063.0,0.081593
3_ребенка,303.0,27.0,0.089109
1_ребенок,4410.0,445.0,0.100907
2_ребенка,1858.0,194.0,0.104413
4_ребенка,37.0,4.0,0.108108
Табор,68.0,8.0,0.117647


**Вывод** после сведения таблицы по количеству детей и просрочкам выяснилось, что самый низкий процент просрочек среди бездетных семей, а самые частые задолжности среди таборов. Чётко видно, что чем больше детей, тем сложнее справиться семье с финансовой нагрузкой. Исключение составляют семьи с пятью детьми, но это связано с маленькой выборкой, что должники не попались, а так же тремя детьми, тут возможно семье помагают финансовые программы государства.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [11]:
# создадим сводную таблицу для оценки задержек в зависимости от семейного статуса
data_pivot_family_status = data.pivot_table(index = ['family_status'], columns = 'debt', values = 'purpose', aggfunc='count')
print(data_pivot_family_status)
print()

# заменим названия столбцов, а то питон на них ругается при подсчете
data_pivot_family_status.set_axis(['no','yes'],axis='columns',inplace=True)
data_pivot_family_status['percentage of delay'] = data_pivot_family_status['yes'] / data_pivot_family_status['no']

data_pivot_family_status.sort_values(by='percentage of delay')

debt                       0    1
family_status                    
Не женат / не замужем   2536  274
в разводе               1110   85
вдовец / вдова           896   63
гражданский брак        3763  388
женат / замужем        11408  931



,no,yes,percentage of delay
family_status,,,
вдовец / вдова,896,63,0.070312
в разводе,1110,85,0.076577
женат / замужем,11408,931,0.081609
гражданский брак,3763,388,0.103109
Не женат / не замужем,2536,274,0.108044


**Вывод** после сведения таблицы к данным столбцов о семейном статусе и наличия просрочек выяснилось, что просрочки чаще встречаются среди холостяков, а самые ответственные вдовцы и вдовы. 

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [12]:
# создадим сводную таблицу для оценки задержек в зависимости от семейного дохода
data_pivot_level_income = data.pivot_table(index = ['level_income'], columns = 'debt', values = 'purpose', aggfunc='count')
print(data_pivot_level_income)
print()

# заменим названия столбцов, а то питон на них ругается при подсчете
data_pivot_level_income.set_axis(['no','yes'],axis='columns',inplace=True)
data_pivot_level_income['percentage of delay'] = data_pivot_level_income['yes'] / data_pivot_level_income['no']

data_pivot_level_income.sort_values(by='percentage of delay')

debt              0    1
level_income            
Высокие        3185  235
Выше среднего  4917  437
Низкие         4277  367
Средние        7334  702



,no,yes,percentage of delay
level_income,,,
Высокие,3185,235,0.073783
Низкие,4277,367,0.085808
Выше среднего,4917,437,0.088875
Средние,7334,702,0.095719


**Вывод** после сведения таблицы по уровню дохода и наличию просрочек выяснилось, что высокие доходы сильно способствуют отсутствию просрочек, а самые частые просрочки, как не странно, среди людей со средним доходом.

- Как разные цели кредита влияют на его возврат в срок?

In [13]:
# создадим сводную таблицу для оценки задержек в зависимости от целей кредита
data_pivot_lemmas = data.pivot_table(index = ['lemmas'], columns = 'debt', values = 'purpose', aggfunc='count')
print(data_pivot_lemmas)
print()

# заменим названия столбцов, а то питон на них ругается при подсчете
data_pivot_lemmas.set_axis(['no','yes'],axis='columns',inplace=True)
data_pivot_lemmas['percentage of delay'] = data_pivot_lemmas['yes'] / data_pivot_lemmas['no']

data_pivot_lemmas.sort_values(by='percentage of delay')

debt              0    1
lemmas                  
автокредит     3903  403
ипотека        7659  587
коммерция      1212   99
образование    3643  370
свадьба        2138  186
строительство  1158   96



,no,yes,percentage of delay
lemmas,,,
ипотека,7659,587,0.076642
коммерция,1212,99,0.081683
строительство,1158,96,0.082902
свадьба,2138,186,0.086997
образование,3643,370,0.101565
автокредит,3903,403,0.103254


**Вывод**  после сведения таблицы к столбцам по цели кредита и наличию просрочки выяснилось, что ответственнее к выплатам относяться при ипотеке и коммерческой недвижимости, а чаще всего просрочки встречаются при автокредите и кредите на образование. Скорее всего это ещё связано и с возрастом, ведь образование и новые автомобили чаще желают молодые люди, которые ещё не привыкли вести свой бюджет.

## Шаг 4. Общий вывод

**Общий вывод**
после ответа на вопросы о зависимости наличия просрочек и разных факторов выяснилось, что сильнее всего это зависит от семейного статуса и целей кредита, разброс в количестве просрочек среди них оказался самым сильным. Холостой человек берущий деньги на автомобиль самый подозрительный клиент. Уверен связано это так же и с его возрастом, ведь молодые люди скорее всего менее ответствено подходят к кредитам. Следом за ними идут семьи с детьми, оно и логично, при наличие детей тежелее расчитать свой бюджет и просрочки встречаются чаще, и чем больше детей, тем тяжелее. А вот уровень дохода, как ни странно оказался с самым маленьким уровнем влияния на вероятность просрочек, хотя конечно люди с высоким достатком испытывают меньше проблем по выплатам. Лучшие клиенты для банка это вдовец с высоким доходом без детей с целью кредита на ипотеку. Необходимо провести анализ на наличие просрочек по возрасту клиентов и их образованию.

Анализ был провёд на основе данных об: струдовом стаже, уровне дохода, семейного статуса, количеству детей, целей кредита и как эти факторы соотносятся с наличием просрочек. Были встречены пропуски в стаже и доходе, где скорее всего люди не отмечали их по причине неофициального трудоустройства. Так же семьи с тремя детьми неожиданно лучше справлялись с кредитами, чем семьи с одним ребенком. Думаю тут "виновато" государство. Так же хоть и выяснилось очевидное, что с высокими доходами людям легче с выплатами по кредитам, но неожиданно семьи с низкими доходами чуть лучше справляються, чем даже семьи, получающие выше среднего. Думаю это связано с более тщательным планированием бюджета.

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [X]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.